# 🛠️ Part 2: Giving Your Agent Custom Tools
## Content Creation Studio Workshop

Welcome back to our series on Google's Agent Development Kit! In Part 1, we built our
first simple agent and saw how it could use a built-in tool like Google Search. That's
powerful, but the real magic begins when we give our agents their own unique abilities.

Today, we're leveling up by creating **custom tools**. This is how you connect an agent
to your own functions, data, or APIs, transforming it from a general researcher into a
specialized content analyst.

---

## 📚 The Content Creation Studio Playbook Series
- Part 1: Building Your First AI Agent ✅
- **Part 2: Giving Your Agent Custom Tools** (You are here) 🎯
- Part 3: Building Agent Teams with Agent-as-a-Tool
- Part 4: Multi-Step Workflows with SequentialAgent
- Part 5: Building Iterative Workflows with LoopAgent
- Part 6: Efficient Workflows with ParallelAgent
- Part 7: The Capstone Project

---

## 🎓 New Concepts in This Part

In Part 2, we'll introduce these **new ADK concepts**:

1. **Custom Function Tools** - Python functions that become agent tools
2. **Tool Signatures & Docstrings** - How agents understand what tools do
3. **The Specialist Agent Pattern** - One agent, one domain of expertise
4. **Tool Limitations** - ADK constraints on mixing tool types

Each concept will be clearly marked with 🆕 when first introduced!

---

## What Are Tools in ADK?

According to the official ADK documentation, tools are how agents interact with the outside
world. They fall into two main categories:

1. **Built-in Tools**: Pre-made tools provided by the ADK for common Google services
   (like Google Search, which we used in Part 1)

2. **Custom Tools**: Tools that you create from your own Python functions. This allows
   you to give your agent any capability you can code!

📖 **Documentation**: [ADK Tools Overview](https://google.github.io/adk-docs/tools/)

## ⚙️ 1. Setup: Install Libraries

First, let's install the Google Agent Development Kit (ADK).

In [ ]:
!pip install google-adk==1.8.0 -q

## 🔑 2. Authentication: Configure Your API Key

Securely provide your Google API key.

In [ ]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

## 🛠️ 3. Creating Custom Tools

### 🆕 NEW CONCEPT: Custom Function Tools

> **What are Custom Function Tools?**  
> Custom function tools are standard Python functions that ADK automatically converts into agent tools. The framework uses the function's name, type hints, and docstring to understand what the tool does and when to use it.
>
> 📖 **Reference**: [Custom Function Tools](https://google.github.io/adk-docs/tools/)

Our goal is to create a **Content Analyzer Agent** that can evaluate blog posts and articles.
To do this, we'll create three custom tools from simple Python functions:

1. **`count_words`** - Counts words in a text
2. **`calculate_readability_score`** - Calculates how easy the text is to read (Flesch Reading Ease)
3. **`generate_hashtags`** - Creates social media hashtags from content

---

### 🆕 NEW CONCEPT: Tool Signatures & Docstrings

> **How Does ADK Understand Tools?**  
> The ADK agent examines three key aspects of your function:
> 1. **Function name** - Identifies the tool
> 2. **Type hints** - Understands parameters (e.g., `text: str`, `count: int`)
> 3. **Docstring** - Learns what the tool does and when to use it
>
> **Best Practice**: Write clear, descriptive docstrings that explain the tool's purpose and usage.
>
> 📖 **Reference**: [Tool Performance Guide](https://google.github.io/adk-docs/tools-custom/performance/)

### Example Tool Structure:
```python
def tool_name(parameter: type) -> return_type:
    """
    Clear description of what this tool does.
    Include when the agent should use it.
    """
    # Implementation
```

**A clear docstring is crucial!** The agent relies on it to decide when to use each tool.

In [ ]:
import re
from typing import List

# Tool 1: Word Counter
def count_words(text: str) -> int:
    """
    Counts the number of words in the provided text.
    Use this when you need to know the length of content.
    """
    print(f"🔧 Tool executed: Counting words in text...")
    words = text.split()
    count = len(words)
    print(f"   Found {count} words")
    return count

def count_syllables(word: str) -> int:
    """Helper function to estimate syllables in a word"""
    word = word.lower()
    vowels = "aeiouy"
    syllable_count = 0
    previous_was_vowel = False

    for char in word:
        is_vowel = char in vowels
        if is_vowel and not previous_was_vowel:
            syllable_count += 1
        previous_was_vowel = is_vowel

    # Adjust for silent e
    if word.endswith('e'):
        syllable_count -= 1

    # Ensure at least 1 syllable
    return max(1, syllable_count)

# Tool 2: Readability Analyzer
def calculate_readability_score(text: str) -> dict:
    """
    Calculates a readability score for the text based on average sentence length
    and average word length. Returns a score from 0-100 where higher is easier to read.

    Use this to evaluate if content is appropriate for the target audience.
    """
    print(f"🔧 Tool executed: Calculating readability score...")

    # Simple readability calculation (simplified Flesch Reading Ease)
    sentences = text.split('.')
    sentences = [s.strip() for s in sentences if s.strip()]

    if not sentences:
        return {"score": 0, "grade": "Unable to calculate", "recommendation": "Add more content"}

    words = text.split()
    total_words = len(words)
    total_sentences = len(sentences)
    total_syllables = sum(count_syllables(word) for word in words)

    # Flesch Reading Ease formula (simplified)
    if total_words == 0 or total_sentences == 0:
        score = 0
    else:
        score = 206.835 - 1.015 * (total_words / total_sentences) - 84.6 * (total_syllables / total_words)
        score = max(0, min(100, score))  # Clamp between 0-100

    # Interpret score
    if score >= 90:
        grade = "Very Easy (5th grade)"
        recommendation = "Great for general audience"
    elif score >= 80:
        grade = "Easy (6th grade)"
        recommendation = "Good for most readers"
    elif score >= 70:
        grade = "Fairly Easy (7th grade)"
        recommendation = "Acceptable for general audience"
    elif score >= 60:
        grade = "Standard (8th-9th grade)"
        recommendation = "Good for average readers"
    elif score >= 50:
        grade = "Fairly Difficult (10th-12th grade)"
        recommendation = "Consider simplifying for broader appeal"
    else:
        grade = "Difficult (College level)"
        recommendation = "Simplify for general audience"

    result = {
        "score": round(score, 2),
        "grade": grade,
        "recommendation": recommendation
    }

    print(f"   Score: {result['score']} - {result['grade']}")
    return result

# Tool 3: Hashtag Generator
def generate_hashtags(text: str, count: int = 5) -> List[str]:
    """
    Generates relevant hashtags from the provided text by extracting key terms.

    Parameters:
    - text: The content to analyze
    - count: Number of hashtags to generate (default: 5)

    Use this to create social media hashtags for content promotion.
    """
    print(f"🔧 Tool executed: Generating {count} hashtags...")

    # Remove common words (stop words)
    stop_words = {
        'the', 'is', 'at', 'which', 'on', 'a', 'an', 'as', 'are', 'was', 'were',
        'been', 'be', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would',
        'could', 'should', 'may', 'might', 'must', 'can', 'of', 'to', 'for', 'in',
        'with', 'by', 'from', 'up', 'about', 'into', 'through', 'during', 'and',
        'or', 'but', 'if', 'then', 'than', 'so', 'this', 'that', 'these', 'those'
    }

    # Extract words and clean them
    words = re.findall(r'\b[a-zA-Z]{4,}\b', text.lower())

    # Filter out stop words and count frequency
    word_freq = {}
    for word in words:
        if word not in stop_words:
            word_freq[word] = word_freq.get(word, 0) + 1

    # Sort by frequency and get top N
    sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    top_words = [word for word, freq in sorted_words[:count]]

    # Convert to hashtag format
    hashtags = [f"#{word.capitalize()}" for word in top_words]

    print(f"   Generated: {', '.join(hashtags)}")
    return hashtags

print("✅ Custom tools defined!")

## 🤖 4. Create the Specialist Agent

### 🆕 NEW CONCEPT: The Specialist Agent Pattern

> **What is the Specialist Agent Pattern?**  
> Instead of creating one "mega-agent" that does everything, the best practice is to build a **team of specialist agents**, where each agent is an expert at a single domain with related tools.
>
> **Benefits**:
> - **Focused**: Each agent has a clear, single responsibility
> - **Maintainable**: Easy to test and debug individual agents
> - **Scalable**: Add new specialists without affecting existing ones
> - **Composable**: Specialists can work together as a team (Part 3!)
>
> **Pattern**: One agent, one domain, related tools

Now we create the `content_analyzer_agent`, whose **only responsibility** is to be an expert
at analyzing content using its set of custom tools.

---

### ⚠️ IMPORTANT: ADK Tool Limitation

> **Limitation**: ADK does not support mixing `google_search` (built-in tool) with custom Python function tools in the same agent.
>
> **Solution**: Create separate specialist agents:
> - `topic_research_agent` → Uses `google_search`
> - `content_analyzer_agent` → Uses custom tools
>
> **In Part 3**, we'll learn how to make these specialists work together!

### What This Agent Does:
When given content, it will:
1. Count the words
2. Calculate readability score
3. Generate hashtags for social media
4. Provide comprehensive feedback

In [ ]:
from google.adk.agents import Agent

content_analyzer_agent = Agent(
    name="content_analyzer_agent",
    model="gemini-2.5-flash",
    description="A specialized agent for analyzing blog content quality, readability, and SEO potential.",
    instruction="""
    You are a content analysis expert. Your only job is to analyze text content and provide
    detailed feedback on its quality.

    When given content, you should:
    1. Use the `count_words` tool to determine content length
    2. Use the `calculate_readability_score` tool to assess readability
    3. Use the `generate_hashtags` tool to create 5 relevant hashtags for social media

    After using your tools, provide a comprehensive analysis:
    - Is the word count appropriate for a blog post? (Aim for 800-2000 words)
    - Is the readability appropriate for the target audience?
    - Are there any specific improvements to suggest?
    - Present the hashtags for social promotion

    You do not research topics or write content. Stick strictly to analysis.
    Be constructive and specific in your feedback.
    """,
    tools=[count_words, calculate_readability_score, generate_hashtags]  # 🔧 Custom tools only
)

print(f"🧞 Specialist Agent '{content_analyzer_agent.name}' is created and ready!")

## 🚀 5. Build the Execution Engine

This is our helper function for running queries. It handles the core ADK logic:
initializing the Runner, streaming events with `run_async`, and displaying the final response.

**Note**: This is the same pattern we learned in Part 1!

In [ ]:
from IPython.display import display, Markdown
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

# Initialize Session Service
session_service = InMemorySessionService()
user_id = "adk_content_creator_001"

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

print("✅ Execution engine ready!")

## ✨ 6. Test the Content Analyzer

Now let's test our specialist agent with a sample blog post! The agent will use all three
custom tools to analyze the content and provide comprehensive feedback.

Watch how the agent:
1. Decides which tools to call
2. Calls each tool and receives results
3. Synthesizes the results into actionable feedback

In [ ]:
# Sample blog content to analyze
sample_blog_content = """
Artificial Intelligence is transforming how small businesses operate in 2025.
From customer service chatbots to inventory management systems, AI tools are
becoming more accessible and affordable than ever before. Small business owners
can now leverage the same technologies that were once exclusive to large corporations.

Machine learning algorithms can analyze customer behavior patterns and predict
future trends with remarkable accuracy. This allows businesses to make data-driven
decisions about inventory, marketing campaigns, and customer engagement strategies.

The key to success is starting small. Begin with one AI tool that addresses your
most pressing business challenge. Whether it's automating email responses or
optimizing your social media posting schedule, there's an AI solution available.

As technology continues to evolve, the gap between small and large businesses
continues to narrow. The future belongs to those who embrace these tools today.
"""

async def run_content_analyzer():
    """Test the content analyzer agent"""
    analyzer_session = await session_service.create_session(
        app_name=content_analyzer_agent.name,
        user_id=user_id
    )

    query = f"Please analyze this blog post:\n\n{sample_blog_content}"
    print(f"📝 Analyzing blog content...")

    await run_agent_query(content_analyzer_agent, query, analyzer_session, user_id)

# Run the analyzer
await run_content_analyzer()

## 🎉 Recap: What We've Learned

This was a huge step forward in understanding agent design:

### Core Concepts Introduced:

1. **🆕 Custom Function Tools** - Python functions that ADK converts to agent tools  
   📖 [Custom Function Tools](https://google.github.io/adk-docs/tools/)

2. **🆕 Tool Signatures & Docstrings** - How agents understand tool behavior  
   📖 [Tool Performance Guide](https://google.github.io/adk-docs/tools-custom/performance/)

3. **🆕 The Specialist Agent Pattern** - One agent, one domain, related tools  
   Best Practice: Build teams of specialists vs mega-agents

4. **⚠️ ADK Limitation** - Cannot mix `google_search` with custom function tools  
   Solution: Create separate specialist agents

### Key Takeaways:

- **Custom tools** are Python functions with clear docstrings
- **Specialist agents** focus on a single domain with related tools
- **Tool design** matters - agents rely on docstrings to choose tools
- **Architecture** guides you toward better design patterns

---

## 🚀 What's Next?

We've successfully created a new specialist agent with powerful custom tools, but this
leaves us with a new question: We now have **two separate agents** - our `topic_research_agent`
from Part 1 and our `content_analyzer_agent`. How do we make them work together as a team?

In **Part 3**, we will solve this exact problem by building an **Orchestrator** and learning
about the powerful **Agent-as-a-Tool** pattern.

### Preview of Part 3 Concepts:
- 🆕 AgentTool Wrapper
- 🆕 The Orchestrator Pattern
- 🆕 Multi-Agent Coordination

See you in Part 3! 🚀